In [1]:
from astropy.io import fits
import astropy.stats
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import csv

from lsst.ts import salobj
from lsst_efd_client import EfdClient, resample
from astropy.time import Time, TimeDelta

In [2]:
d = salobj.Domain()

cbp = salobj.Remote(d, 'CBP')
await cbp.start_task

In [3]:
# Check heartbeat
tmp = await cbp.evt_heartbeat.next(flush=True,timeout=5)
print(tmp)

private_revCode: f7344f34, private_sndStamp: 1666126884.3589225, private_rcvStamp: 1666126884.360867, private_seqNum: 1159558, private_identity: CBP, private_origin: 4738, heartbeat: False


In [4]:
tmp = await cbp.cmd_setLogLevel.set_start(level=logging.DEBUG)
print(tmp)

private_revCode: 088d54ca, private_sndStamp: 1660593914.9751272, private_rcvStamp: 1660593914.9772367, private_seqNum: 1516984070, private_identity: CBP, private_origin: 31608, ack: 303, error: 0, result: Done, identity: parfa30@nb-parfa30, origin: 9575, cmdtype: 10, timeout: 0.0


In [5]:
tmp = await cbp.evt_softwareVersions.aget(timeout=4)
print(tmp)

private_revCode: d31f44a3, private_sndStamp: 1660588633.6157324, private_rcvStamp: 1660593909.159937, private_seqNum: 1, private_identity: CBP, private_origin: 31608, salVersion: 7.0.0, xmlVersion: 12.0.0, openSpliceVersion: 6.10.4p2, cscVersion: ?, subsystemVersions: 


In [6]:
tmp = await cbp.evt_configurationsAvailable.aget(timeout=5)
print(tmp)

private_revCode: 808d2b14, private_sndStamp: 1660588633.6532998, private_rcvStamp: 1660593909.1383848, private_seqNum: 1, private_identity: CBP, private_origin: 31608, overrides: , version: heads/tickets/DM-34387-0-gf0b0e35, url: file:///home/saluser/develop/ts_config_mtcalsys/CBP/v1, schemaVersion: v1


In [8]:
config=""

In [113]:
# grab timestamp for EFD queries for debugging
t2 = Time(Time.now(), scale='utc')

In [15]:
#get summary state
tmp = await cbp.evt_summaryState.aget(timeout=5)
print(salobj.State(tmp.summaryState))

State.ENABLED


In [106]:
#  bring into standby state if required
#await salobj.set_summary_state(cbp, salobj.State.OFFLINE, override=config)

[<State.ENABLED: 2>,
 <State.DISABLED: 1>,
 <State.STANDBY: 5>,
 <State.OFFLINE: 4>]

In [16]:
await cbp.cmd_standby.set_start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1640943334, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: standby not allowed in state <State.ENABLED: 2>')

In [7]:
#  bring into enable state if required
await salobj.set_summary_state(cbp, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [56]:
tmp = await cbp.cmd_standby.set_start()

In [57]:
tmp = await cbp.cmd_start.set_start()
# print(tmp)

In [14]:
tmp = await cbp.cmd_changeMask.set_start(mask=2)
print(tmp)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=157082181, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result="Failed: 'NoneType' object has no attribute 'name'")

In [18]:
# Get the target position
target = await cbp.evt_target.aget(timeout=2)
print(target)


private_revCode: 556da9fb, private_sndStamp: 1666127614.6862218, private_rcvStamp: 1666127614.6878915, private_seqNum: 9, private_identity: CBP, private_origin: 4738, elevation: 6.0, azimuth: 6.0, focus: 5002.56, mask: mask 1, mask_rotation: 359.9789


In [17]:
in_position = await cbp.evt_inPosition.aget(timeout=2)
print(in_position)

private_revCode: f2728ab9, private_sndStamp: 1666127617.96509, private_rcvStamp: 1666127617.966304, private_seqNum: 22, private_identity: CBP, private_origin: 4738, elevation: True, azimuth: True, focus: True, mask: True, mask_rotation: False


In [91]:
import numpy as np

In [ ]:
cbp.cmd_

In [21]:
dp=np.random.random(1)*5
print(dp)
az=5 #+ dp
el=-4#+ dp
print(f'az={az}, el={el}')
tmp=await cbp.cmd_move.set_start(azimuth=az, elevation=el, timeout=25)
print(tmp)

[2.01853]
az=5, el=-4
private_revCode: 088d54ca, private_sndStamp: 1660692281.5169468, private_rcvStamp: 1660692281.518392, private_seqNum: 864261564, private_identity: CBP, private_origin: 26274, ack: 303, error: 0, result: Done, identity: parfa30@nb-parfa30, origin: 10805, cmdtype: 6, timeout: 0.0


In [16]:
tmp = await cbp.cmd_move.set_start(azimuth=6, elevation=6)
print(tmp)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=750971398, ack=<SalRetCode.CMD_TIMEOUT: -304>, error=1, result='Timeout')

In [10]:
tmp = await cbp.cmd_setFocus.set_start(focus=5000, timeout=30)
print(tmp)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1290409312, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result="Failed: could not convert string to float: ''")

In [10]:
# change the mask
await cbp.cmd_changeMask.set_start(mask="1", timeout=60)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=157082180, ack=<SalRetCode.CMD_TIMEOUT: -304>, error=1, result='Timeout')

In [19]:
# rotate the mask
await cbp.cmd_changeMask.set_start(mask="3")

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=157082182, ack=<SalRetCode.CMD_TIMEOUT: -304>, error=1, result='Timeout')